July 28, 2016 - Women in Data Science Meetup - "Data Science from Scratch" Workshop #5

# Data Science from Scratch Tutorial
## Naive Bayes
### Julia Galstad 2016

The goal of this notebook is to understand the Naive Bayes classifier found in chapter 13 of 'Data Science from Scratch' by Joel Grus. This project is creating a baby spam filter using the data from the SpamAssassin Public Corpus. https://spamassassin.apache.org/publiccorpus/

To help familiarize with concepts and notation, there are some problems at the beginning that can be done 'by hand.' Please skip as much as you'd like.

At the end of this notebook, there is a brief discussion of the distributions commonly used with Naive Bayes, and a small reference to one with a Guassian normal distribution to provide contrast.

### Import Libraries

In [1]:
from __future__ import division                   
from collections import defaultdict, Counter      
import re                                   # for 'regular expression' operations
import glob                                 # for work with pathname patterns
import math, random
import notebook_answers as ans              # not a standard python module
                                                # contains hints and answers

## Conditional Probability and Bayes' Theorem
### Cookie example

Let's practice applying Bayes' Theorem by solving the cookie problem formally found on <a href="https://en.wikipedia.org/wiki/Bayes%27_theorem" target="_blank">this</a> Wikipedia page . (Thank you to Allen Downey for preserving it.)

There are two bowls of cookies:

    Bowl #1 has 30 oatmeal cookies and 10 chocolate cookies. 
    
    Bowl #2 has 20 oatmeal cookies and 20 chocolate cookies.
    
Someone gets an oatmeal cookie. What is the probability it is from Bowl #1?
    

### Before solving this question, let's calculate some other helpful probabilities.

### Step 1

### <font color='blue'>What is $P($Bowl #1$)$, the probability a cookie is from Bowl #1, before knowing it was oatmeal or chocolate? </font>

In [4]:
# You can get a hint or the answer if you choose 
# and/or use the python notebook as your calculator.
print(ans.hint(1))
#print(ans.answer(1))

### Step 2

### <font color='blue'>What is $P($oatmeal$\mid$Bowl #1$)$, the probability a cookie is oatmeal given that it came from Bowl #1? </font>

In [6]:
print(ans.hint(2))
#print(ans.answer(2))

### Step 3

### <font color='blue'>What is $P($oatmeal$)$, the probability a cookie is oatmeal? </font>

This question can be solved in two ways. 

Let's practice using the Law of Total Probability: $$P(E) = P(H)P(E\mid H) + P(\neg H)P(E\mid \neg H).$$

In our cookie example you would calculate:

$$P({\rm oatmeal}) = P({\rm Bowl}\,{\rm \#1})P({\rm oatmeal}\mid {\rm Bowl}\,{\rm \#1 }) + P({\rm Bowl}\,{\rm\#2})P({\rm oatmeal }\mid{\rm Bowl}\,{\rm \#2 }).$$

In [7]:
print(ans.hint(3))
#print(ans.answer(3))

### Now we're nearly ready to put it all together to answer the main question:


### <font color='blue'>What is $P($ Bowl #1$\mid$oatmeal $)$, the probability that the cookie was from Bowl #1, given that it's oatmeal? </font>

### Let's use Bayes' Theorem to solve this. 

We can state Bayes' Theorem as

$$ P(H \mid E) = \frac{P(H)P(E \mid H)}{P(E)}$$

where $H$ is the hypothesis, and $E$ is the evidence.



In this example, the hypothesis, $H$, is:

    'the cookie is from Bowl #1' 
and the evidence, $E$, is:

    'the cookie is oatmeal.' 
    
We could write out Bayes' Theorem for our example like this:
$$P({\rm Bowl}\,{\rm \#1} \mid{\rm oatmeal}) = \frac{P({\rm Bowl}\,{\rm \#1})P({\rm oatmeal}\mid {\rm Bowl}\,{\rm \#1 })}{P({\rm oatmeal})}.$$

Recall, we want to solve for $P($ Bowl #1$\mid$oatmeal $)$, the probability that the cookie was from Bowl #1, given that it's oatmeal.

We've already calculated the values we need to plug into Bayes' Theorem, so now let's do just that: substitute the values into the right hand side of the formula.

In [5]:
print(ans.hint(4))
#print(ans.answer(4))

### Bayesian interpretation: accounting for new evidence

'In the Bayesian (or epistemological) interpretation, probability measures a degree of belief. Bayes' theorem then links the degree of belief in a proposition before and after accounting for evidence.' (<a href="https://en.wikipedia.org/wiki/Bayes%27_theorem" target="_blank">Wikipedia</a>)

In the cookie example, our original hypothesis was the belief that the cookie was from Bowl #1. We then found new evidence, that it was oatmeal. We then updated our belief to include this new information using Bayes' Theorem.

### <font color='blue'>What kind of evidence do you think makes sense to use to determine if an email is spam or not? </font>

You may discuss your ideas with your group and/or post them to the slack channel.

## Building Intuition towards a Naive Bayes spam classifier
### Using words as evidence

Our spam filter will use the occurence of a word in a message as evidence.

Specifically, if our training set emails contain $n$ distinct words, $w_1, \dots, w_n$, we will have $n$ features/events $X_1, \dots, X_n$, where $X_i$ stands for 'a message contains the word $w_i$.' We are assuming (which is false, as described in the talk) that the features are independent. 

We call the set of words $w_1, \dots, w_n$ a $vocabulary$. 

For each word $w_i$, we calculate conditional probabilities of that word occuring in a spam message and in a non-spam message, $P(X_i\mid S)$ and $P(X_i\mid \neg S)$, respectively, where $S$ is spam.

To calculate this values, we need to determine which words occur in which emails.

### Tokenizing messages

Given some email data set, we want to $tokenize$ each message. For us, that means to turn each message into a collection of unique lowercase words that the message was comprised of. (If you're familiar with NLP, you know you would want to throw out function words like $of$, $the$. Our simple ``tokenize`` function doesn't do that.) 

The function ``tokenize`` from Joel Grus is below.

In [27]:
def tokenize(message):
    message = message.lower()                       # convert to lowercase
    all_words = re.findall("[a-z0-9']+", message)   # extract the words, can also use \w as the filter
    return set(all_words)                           # remove duplicates

### Example use of ``re.findall``

The first argument, ``[a-z0-9']+``, that we pass to ``findall`` is a <a href="https://en.wikipedia.org/wiki/Regular_expression" target="_blank">regular expression</a>, which is a specific syntax that represents a pattern that we want our text to match. 

Using ``[a-z0-9']+`` indicates we are looking for words that contain the symbols: ``a-z0-9'``. 

The ``+`` symbol indicates we want words and not just letters.

In [34]:
re.findall("[a-z0-9]", "abc's:of testING;")

In [33]:
re.findall("[a-z0-9']+", "abc's:of testING;42Sup 42sup")

### Example use of ``tokenize``

In [32]:
tokenize('"We should start back", Gared urged as the woods began to grow'
         'dark around them. "The wildlings are dead."')

### <font color='blue'>Just for fun -- Who wrote these sentences? </font>

In [4]:
print(ans.answer(9))

### Counting words

In order to estimate conditional probabilities, we count the occurences of the words.

The function ``count_words`` by Joel Grus tokenizes a message and then 'return a dictionary whose keys are words, and whose values are two-element lists ``[spam_count, non_spam_count]`` corresponding to how many times we saw that word in both spam and nonspam messages.'

In [5]:
def count_words(training_set):
    """training set consists of pairs (message, is_spam)"""
    counts = defaultdict(lambda: [0, 0])
    for message, is_spam in training_set:
        for word in tokenize(message):
            counts[word][0 if is_spam else 1] += 1
    return counts

### A toy example -- from ``count_words`` to computing conditional probabiities

Let's pretend that we just ran ``count_words`` on a small training set (with 400 spams and 425 non-spams), and returned this dictionary, which we saved as ``winter_dict``.

The dictionary keys are the words of our vocabulary.
The dictionary value associated with its key (a word) is a list that contains the total number of spam emails in which that word occured and the total number of non-spam emails in which that word occured. 

``{``'``word``': [``spam_count``, ``non_spam_count``]``}``

In [14]:
winter_dict = {'hodor': [100, 400], 'winter': [50, 20], 'witch': [300, 10], 'red': [10, 100], 'dragon': [0, 10]}

### Let's think about how to calculate the conditional probabilities we'll need for Naive Bayes using a dictionary like this. Take 5 minutes to answer three questions about ``winter_dict``.

### <font color='blue'>1. How many words are in our vocabulary? What are they? </font>

In [18]:
print(ans.answer(10))

### <font color='blue'>2. Using the dictionary ``winter_dict``, estimate $P(X_1 \mid S)$. That is, estimate $P($ 'hodor' $\mid$ spam $)$, the probability that a message has the word ``'hodor'``, given that we know it is spam. </font>

In [7]:
print(ans.hint(11))
#print(ans.answer(11))

#### (We're using the symbol $\neg$ to indicate negation. This way we can write 'not spam' as $\neg S$.)

### <font color='blue'>3. Using the dictionary ``winter_dict``, estimate $P(X_1 \mid \neg S)$. That is, estimate $P($ 'hodor' $\mid$ not spam $)$, the probability that a message has the word ``'hodor'``, given that we know the message is not spam. </font>

In [16]:
print(ans.hint(12))
#print(ans.answer(12))

### Now that we've calculated these conditional probabilities that we'll need for Naive Bayes, we can make a function that calculates them automatically.


We define a function ``word_probabilities_prototype`` that assigns conditional probabilites to words in our vocabulary:

In [7]:
def word_probabilities_prototype(counts, total_spams, total_non_spams):
    """Turn the word_counts into a list of triplets
    word, p(word | spam) and p(word | not spam)"""
    return [(word,
             spam / total_spams ,
             non_spam / total_non_spams)
             for word, (spam, non_spam) in counts.iteritems()]

Let's apply the function ``word_probabilities_prototype`` to our toy dictionary ``winter_dict``:

In [16]:
winter_word_probabilities = word_probabilities_prototype(winter_dict, 400, 425)
print(winter_word_probabilities)

(A word of caution: Dictionaries don't preserve order.

When ``word_probabilities_prototype`` iterates through key-value pairs in ``winter_dict``, the word associated with the second feature might not appear second in the list ``winter_word_probabilities``. 

In fact, for me, I decided the word associated with feature $X_2$ is `'winter'`, but ``winter_words_probabilties[2]=('winter', 0.125, 0.047058823529411764)``.)

### Now let's classify some messages!

We are going to start off with the message ``'Hodor Hodor Bran'``, which in our test set isn't spam.

Remember our features are the presence (or absence) of our vocabulary words in the message, so the fact that ``'Hodor'`` appears twice, or the fact that the word ``'Bran'`` appears at all, are irrelevant.

### What features represent this message?

Thinking about the vocabulary of our model, the probability that the message ``'Hodor Hodor Bran'`` is spam (and this is an abuse of notation here) is:
$$P({\rm Spam }\mid {\rm hodor = True, winter = False, witch = False, red = False, dragon = False})$$

In appropriate notation, this is the probability we are estimating:
$$P(S \mid X_1, \neg X_2, \neg X_3, \neg X_4, \neg X_5)$$

As shorthand, you could write: $$P(S|E)$$ where $E$ stands for the evidence described by our features $X_1, \neg X_2, \neg X_3, \neg X_4, \neg X_5$.

We then apply Bayes' Theorem to get:
$$P(S \mid E) = \frac{P(S)P(E\mid S)}{P(E)}$$

### Probability of Spam Assumption

We'll assume that a message is equally likely to be spam or not spam, so we set $P(S) = P (\neg S) = 0.5$.

(Most implementations of Naive Bayes don't make this assumption.)

### Law of Total Probability

We use the law of total probability, $P(E) = P(S)P(E\mid S) + P(\neg S)P(E\mid \neg S)$, to transform the denominator. 
$$P(S \mid E) = \frac{P(S)P(E\mid S)}{P(S)P(E\mid S) + P(\neg S)P(E\mid \neg S)}$$

We did this because our assumption that $P(S)=P(\neg S)=0.5$ was helpful; these terms cancel out and can be ignored. 

$$P(S \mid E) = \frac{{\rm \bf{0.5}}\cdot P(E\mid S)}{{\rm \bf{0.5}}\cdot P(E\mid S) + {\rm \bf{0.5}}\cdot P(E\mid \neg S)}$$

$$P(S \mid E) = \frac{{\rm \bf{0.5}}\cdot P(E\mid S)}{{\rm \bf{0.5}}\cdot \left(P(E\mid S) +  P(E\mid \neg S)\right)}$$

$$P(S \mid E) = \frac{P(E\mid S)}{P(E\mid S) + P(E\mid \neg S)}$$

### Chain Rule simplification

Consider the probability $P(E\mid S)$ on the right-hand side. (The simplifications with $P(E\mid \neg S)$ are similar.)

Recall, the evidence from our example message ``'Hodor Hodor Bran'`` was $X_1, \neg X_2, \neg X_3, \neg X_4, \neg X_5$, so $$P(E\mid S)=P(E\mid X_1, \neg X_2, \neg X_3, \neg X_4, \neg X_5).$$


Apply Chain Rule to get:
$$P(E \mid S) = P(X_1 \mid S)P(\neg X_2 \mid X_1,S)P(\neg X_3 \mid X_1, \neg X_2,S)P(\neg X_4 \mid X_1, \neg X_2, \neg X_3,S), P(\neg X_5 \mid X_1, \neg X_2, \neg X_3, \neg X_4,S)$$

### Naive Bayes assumption

With our (false) Naive Bayes assumption that the presence of these words are conditionally independent, we ignore the effects of the other words:

$$P(E \mid S) = P(X_1\mid S)P(\neg X_2\mid S)P(\neg X_3\mid S)P(\neg X_4\mid S)P(\neg X_5 \mid S)$$

Our formula is now: $$P(S \mid E) = \frac{P(E\mid S)}{P(E\mid S) + P(E\mid \neg S)}$$

$$P(S \mid E) = \frac{P(X_1\mid S)P(\neg X_2\mid S)P(\neg X_3\mid S)P(\neg X_4\mid S)P(\neg X_5 \mid S)}{{\rm same} \,{\rm as} \, {\rm numerator} + {\rm similar} \, {\rm to} \, {\rm  numerator}}$$

### <font color='blue'> Our list ``winter_word_probabilities`` contains $P(X_1\mid S) = 0.25$, the probability of ``'hodor'`` given that the message is spam. </font>

### <font color='blue'>Can we use ``winter_word_probabiiltes`` to calculate the other probabilities in the formula, such as $P(\neg X_2 \mid S)$? </font>

### Hint: We'll need to use: $P(\neg X \mid S ) = 1 - P(X \mid S)$

First, we remember that $P(X) + P(\neg X) = 1$. In a more useful form,  $P(\neg X) = 1 - P(X)$. This applies to conditional probability, too: $$P(\neg X \mid S) = 1 - P(X \mid S)$$

Our list contains $P(X_2 \mid S)=0.125$, the probability of ``'winter'`` given that the mesage is spam. 

Therefore: $$P(\neg X_2 \mid S) = 1 - 0.125 = 0.875$$

### <font color='blue'> To check your understanding, use the numbers in ``winter_word_probabilities`` to verify that the numerator of this formula evaluates to $0.25 \cdot (1 - 0.125) \cdot (1 - 0.75) \cdot (1 - 0.025) \cdot (1 - 0)$: </font>

$$P(S \mid E) = \frac{P(X_1\mid S)(1- P(X_2\mid S))(1 -P(X_3\mid S))(1 -P( X_4\mid S))(1 - P(X_5 \mid S))}{{\rm same} \,{\rm as} \, {\rm numerator} + {\rm similar} \, {\rm to} \, {\rm  numerator}}$$


In [17]:
# To help you with the above verification, again we can look at the 
# list titled winter_word_probabilities containing the triples:
# [word, P(word | spam), P(word | not spam)]
print(winter_word_probabilities)

[('hodor', 0.25, 0.9411764705882353), ('witch', 0.75, 0.023529411764705882), ('winter', 0.125, 0.047058823529411764), ('red', 0.025, 0.23529411764705882), ('dragon', 0.0, 0.023529411764705882)]


### This formula that we just verified is going to be the crux of the main function in our Naive Bayes classifier.

To implement this formula, we define the function ``spam_probability_prototype``, which estimates the probability our message is spam given the evidence provided by our features.

In [5]:
def spam_probability_prototype(word_probs, message):
    message_words = tokenize(message)
    message_prob_if_spam = message_prob_if_not_spam = 1.0

    # iterate through each word in our vocabulary
    for word, prob_if_spam, prob_if_not_spam in word_probs:

        # if *word* appears in the message,
        # multiply the probability of seeing it
        if word in message_words:
            message_prob_if_spam *= prob_if_spam
            message_prob_if_not_spam *= prob_if_not_spam

        # if *word* doesn't appear in the message
        # multiply the probability of _not_ seeing it
        # which is (1 - probability of seeing it)
        else:
            message_prob_if_spam *= (1.0 - prob_if_spam)
            message_prob_if_not_spam *= (1.0 - prob_if_not_spam)

    return message_prob_if_spam / (message_prob_if_spam + message_prob_if_not_spam)

### <font color='blue'> Do we classify the message 'Hodor Hodor Bran' as spam or not spam? </font>

In [10]:
print(spam_probability_prototype(winter_word_probabilities, 'Hodor Hodor Bran'))
#print(ans.answer(13))

### Argmax

### <font color='blue'> In the general Naive Bayes algorithm, the classification step uses an argmax function as a part of the decision rule. Why didn't we do that here? </font>

In [11]:
#print(ans.answer(13.5))

In the usual implementation of Naive Bayes, we leave out the denominator of Bayes' Theorem.
### <font color='blue'> If we omit the denomiator, can we still leave out argmax? </font>

In [8]:
print(ans.hint(13.7))
#print(ans.answer(13.7))

### <font color='blue'>How do we classify the message 'witch red winter'? </font>

In [36]:
print(spam_probability_prototype(winter_word_probabilities, 'witch red winter'))
# print(ans.answer(14))

### <font color='blue'> How do we classify the message 'witch red winter dragon'? </font>

In [39]:
print(spam_probability_prototype(winter_word_probabilities, 'witch red winter dragon'))
#print(ans.answer(15))

### <font color='blue'> What happened? Why would adding the word 'dragon' change the probabilities so much? </font>

In [10]:
#print(ans.answer(16))

### <font color='blue'> Why is this a problem? Do you see any other problems with our model? </font>

### Improving our Naive Bayes ideas

To fix this 0-annihilation problem, we will change our word_probabilities_prototype function to include smoothing with a $pseudocount$. It's time to hallucinate!

"When computing the spam probabilities for the $i$th word, we assume we also saw $k$ additional spams containing the word and $k$ additional spams not containing the word." (Joel Grus)

"If 'dragon' occurs in 0/98 spam documents, and if k is 1, we estimate $P($ dragon $\mid$ spam $)$  as 1/100 = 0.01, which allows our classifier to still assign some nonzero spam probability to messages that contain the word 'dragon.'" (a quote of Joel Grus that was dragon-ified)


### The function  ``word_probabilities`` calculates conditional word probabilities using a $pseudocount$:

In [6]:
def word_probabilities(counts, total_spams, total_non_spams, k=0.5):
    """turn the word_counts into a list of triplets 
    w, p(w | spam) and p(w | ~spam)"""
    return [(w,
             (spam + k) / (total_spams + 2 * k),
             (non_spam + k) / (total_non_spams + 2 * k))
             for w, (spam, non_spam) in counts.iteritems()]

### <font color='blue'> Using ``word_probabilities``, how should we classify the message 'witch red winter dragon'? </font>

In [46]:
print(ans.hint(17))
# print(ans.hint(17.1))
# print(ans.answer(17))

### Calculations with small numbers

There's another issue we need to fix in our model. (It's not a problem in our tiny five-word vocabulary example.) Suppose we have a very large vocabulary, which includes some words with extremely high word counts and other words with extremely low word counts. We could be potentially multiplying together lots of very small numbers when we calculate probabilities.

### <font color='blue'> What might go wrong? </font>

'In practice, you usually want to avoid multiplying lots of probabilities together, to avoid a problem called $underflow$, in which computers don’t deal well with floating-point numbers that are too close to zero.' 


'We usually compute a product of probabilities $p_1p_2 \cdots p_n$ as the equivalent (but floating-point-friendlier): $$\exp(\log(p_1)+\log(p_2)+\cdots + \log(p_n)).'$$
(Joel Grus)


This formula comes from combining the facts that the exponential and logarithm functions are inverses: $$\exp(\log(x))=x$$ and that in log space, you can compute multiplications as additions: $$\log(ab)=\log(a)+\log(b).$$


### The updated ``spam_probability`` function that takes $underflow$ into account:

In [7]:
def spam_probability(word_probs, message):
    message_words = tokenize(message)
    log_prob_if_spam = log_prob_if_not_spam = 0.0

    for word, prob_if_spam, prob_if_not_spam in word_probs:

        # for each word in the message, 
        # add the log probability of seeing it 
        if word in message_words:
            log_prob_if_spam += math.log(prob_if_spam)
            log_prob_if_not_spam += math.log(prob_if_not_spam)

        # for each word that's not in the message
        # add the log probability of _not_ seeing it
        else:
            log_prob_if_spam += math.log(1.0 - prob_if_spam)
            log_prob_if_not_spam += math.log(1.0 - prob_if_not_spam)
            
    prob_if_spam = math.exp(log_prob_if_spam)
    prob_if_not_spam = math.exp(log_prob_if_not_spam)
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)

### Putting the pieces together into a classifier:

In [8]:
class NaiveBayesClassifier:

    def __init__(self, k=0.5):
        self.k = k
        self.word_probs = []

    def train(self, training_set):
    
        # count spam and non-spam messages
        num_spams = len([is_spam 
                         for message, is_spam in training_set 
                         if is_spam])
        num_non_spams = len(training_set) - num_spams

        # run training data through our "pipeline"
        word_counts = count_words(training_set)
        self.word_probs = word_probabilities(word_counts, 
                                             num_spams, 
                                             num_non_spams,
                                             self.k)
                                             
    def classify(self, message):
        return spam_probability(self.word_probs, message)

## Testing the Model on SpamAssassin data
### Loading the data

We'll use just the subject lines of the emails from the files prefixed with 20021010.

You'll need to change the code below based on where you've saved the data on your machine.

The command 'pwd' will tell you the current path of this notebook.

If you've saved the three folders (spam, easy_ham, hard_ham) in a 'data' folder, and 'data' is in the same place as this notebook, then putting the path from 'pwd' in front of /data/\*/\* will probably work.

In [ ]:
pwd

In [29]:
# modify the path with wherever you've put the files
# there should be 3 folders: spam, easy_ham, hard_ham
# Below assumes they are stored in one folder named 'data'
# 
# the command 'pwd' will tell you the current path of this notebook and
# might be what you put in front of /data/*/*
# 
path = r"C:\data\*\*"

data = []

# glob.glob returns every filename that matches the wildcarded path
for fn in glob.glob(path):
    is_spam = "ham" not in fn

    with open(fn,'r') as file:
        for line in file:
            if line.startswith("Subject:"):
                # remove the leading "Subject: " and keep what's left
                subject = re.sub(r"^Subject: ", "", line).strip()
                data.append((subject, is_spam))

In [12]:
# Have a look at part of the data
print data[0:5], data[3000:3005]

[('Re: New Sequences Window', False), ('[zzzzteana] RE: Alexander', False), ('[zzzzteana] Moscow bomber', False), ("[IRR] Klez: The Virus That  Won't Die", False), ('Re: Insert signature', False)] [('Re: Adult Classifieds', True), ('Low Price Smokes', True), ('Low Price Tobacco', True), ('REQUEST FOR MUTUALLY BENEFITTING ENDEAVOUR.', True), ("** You're -Approved-!", True)]


### Split the data into training and test data.

In [13]:
def split_data(data, prob):
    """split data into fractions [prob, 1 - prob]"""
    results = [], []
    for row in data:
        results[0 if random.random() < prob else 1].append(row)
    return results

random.seed(0)                                  # just so you get the same answers as Joel
train_data, test_data = split_data(data, 0.75)

### Build a classifier with our training data.

In [14]:
classifier = NaiveBayesClassifier()
classifier.train(train_data)

### Testing the model

Test to see how well the model is done.

In [15]:
# triplets (subject, actual is_spam, predicted spam probability)
classified = [(subject, is_spam, classifier.classify(subject))
              for subject, is_spam in test_data]

# assume that spam_probability > 0.5 corresponds to spam prediction
# and count the combinations of (actual is_spam, predicted is_spam)
counts = Counter((is_spam, spam_probability > 0.5)
                 for _, is_spam, spam_probability in classified)

print(counts)

Counter({(False, False): 704, (True, True): 101, (True, False): 38, (False, True): 33})


'This gives 101 true positives (spam classified as “spam”), 33 false positives (ham classified as “spam”), 704 true negatives (ham classified as “ham”), and 38 false negatives (spam classified as “ham”). This means our precision is 101 / (101 + 33) = 75%, and our recall is 101 / (101 + 38) = 73%, which are not bad numbers for such a simple model.' (Grus)

### Precision and Recall

$Precision$ and $recall$ are common measurements for classification problems. See <a href="https://en.wikipedia.org/wiki/Precision_and_recall" target="_blank"> here </a> for more info. For us, on the image below, the relevant elements are the spam emails.

<img src=images/Precisionrecall.png align="center" width="40%"></div>

### 'It’s also interesting to look at the most misclassified:' (Grus)

In [2]:
# sort by spam_probability from smallest to largest
classified.sort(key=lambda row: row[2])

# the highest predicted spam probabilities among the non-spams
spammiest_hams = filter(lambda row: not row[1], classified)[-5:]

# the lowest predicted spam probabilities among the actual spams
hammiest_spams = filter(lambda row: row[1], classified)[:5]

print(spammiest_hams)
print(hammiest_spams)

'The two spammiest hams both have the words “needed” (77 times more likely to appear in spam), “insurance” (30 times more likely to appear in spam), and “important” (10 times more likely to appear in spam).

The hammiest spam is too short (“Re: girls”) to make much of a judgment, and the second-hammiest is a credit card solicitation most of whose words weren’t in the training set.' (Grus)

### Now check out the 'spammiest' words.

In [17]:
def p_spam_given_word(word_prob):
    """uses bayes's theorem to compute p(spam | message contains word)"""

    # word_prob is one of the triplets produced by word_probabilities
    word, prob_if_spam, prob_if_not_spam = word_prob
    return prob_if_spam / (prob_if_spam + prob_if_not_spam)

words = sorted(classifier.word_probs, key=p_spam_given_word)

spammiest_words = words[-5:]
hammiest_words = words[:5]
print(spammiest_words)
print(hammiest_words)

[('year', 0.028767123287671233, 0.00022893772893772894), ('sale', 0.031506849315068496, 0.00022893772893772894), ('rates', 0.031506849315068496, 0.00022893772893772894), ('systemworks', 0.036986301369863014, 0.00022893772893772894), ('money', 0.03972602739726028, 0.00022893772893772894)]
[('spambayes', 0.0013698630136986301, 0.04601648351648352), ('users', 0.0013698630136986301, 0.036401098901098904), ('razor', 0.0013698630136986301, 0.030906593406593408), ('zzzzteana', 0.0013698630136986301, 0.029075091575091576), ('sadev', 0.0013698630136986301, 0.026785714285714284)]


### <font color='blue'> What ways can you think of to improve this spam filter? </font>

Some suggestions from Grus:

$\cdot$ Modify the classifier to accept an optional min_count threshhold and ignore tokens that don’t appear at least that many times.

$\cdot$ We could add extra features like “message contains a number” by creating phony tokens like contains:number and modifying the tokenizer to emit them when appropriate.

$\cdot$ The tokenizer has no notion of similar words (e.g., “cheap” and “cheapest”). Modify the classifier to take an optional stemmer function that converts words to equivalence classes of words. For example, a really simple stemmer function might be:

In [53]:
def drop_final_s(word):
    return re.sub("s$", "", word)

### <font color='blue'> What issues might come up if you include the whole message and not just the subject? </font>

### A brief word about probability distributions

Distributions describe what kind of outcomes we expect a random variable to take. For more information, check out <a href="http://www.dartmouth.edu/~chance/teaching_aids/books_articles/probability_book/amsbook.mac.pdf" target="_blank"> this</a> open source probability book. 

We make assumptions about these distributions when we design our model. 

Scikit-learn implements <a href="http://scikit-learn.org/stable/modules/naive_bayes.html" target="_blank">three different versions</a> of Naive Bayes, based on either Bernoulli, a Multinomial or Gaussian distribution.

We assumed our data followed a <a href="https://en.wikipedia.org/wiki/Bernoulli_distribution" target="_blank">Bernoulli distribution</a>.

In contrast, <a href="https://en.wikipedia.org/wiki/Naive_Bayes_classifier#Gender_classification" target="_blank">this gender classification example</a> follows a Gaussian distribution. 